# Slide Inference

## Import Packages

In [1]:
from evaluation_helper import *
import sys
sys.path.insert(0, '../segmentation')

## Define model input parameters

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
segmentation_patch_size = 512
classification_patch_size = 1024
batch_size = 2
segmentation_level = 2
classification_level = 0

## Define paths of slide folder and annotation JSON

In [3]:
target_folder = Path("E:/Slides/Canine Skin Tumors")
annotation_file = "E:/Slides/Canine Skin Tumors/skin_tumors.json"

## Define dict to match annotation classes to classes used for training

In [4]:
label_dict = {'Bg': 0, 'Bone': -1, 'Cartilage': -1, 'Dermis': 1, 'Epidermis': 2, 'Subcutis': 3, 'Inflamm/Necrosis': 4,'Melanoma': 5, 'Plasmacytoma': 5,'Mast Cell Tumor': 5, 'PNST': 5, 'SCC': 5, 'Trichoblastoma': 5, 'Histiocytoma': 5}

## Load slide in custom SlideContainer format

In [5]:
slide = SlideContainer(Path(str(target_folder) + "/PNST/1654_16a_PNST_MF.svs"),annotation_file, segmentation_level, segmentation_patch_size, segmentation_patch_size, dataset_type='segmentation', label_dict=label_dict)

## Load learners

In [6]:
segmentation_learner = load_learner(Path("../models"), "ResNet18_512_2_sdata.pkl")
classification_learner = load_learner(Path("../models"), "xception_0_sdata.pkl")

## Slide inference

In [7]:
wsi_inference(slide,device,segmentation_patch_size,classification_patch_size,segmentation_level,classification_level, batch_size, segmentation_learner,classification_learner)

## Confusion Matrix

In [8]:
cm = slide_cm_matrix(slide, segmentation_learner.data.c)

## Segmentation Jaccard score

In [9]:
jaccard_score(cm, segmentation_learner.data.classes)

IoUs:  {'Bg': 0.9614, 'Dermis': 0.7214, 'Epidermis': 0.7731, 'Subcutis': 0.6407, 'Inflamm/Necrosis': 0.7842, 'Tumor': 0.9503} Mean:  0.8051762152594036
Frequency-Weighted IoU:  0.9266


## Classification accuracy

In [10]:
slide_accuracy(slide, classification_learner.data.classes)

Classification Accuracies:  {'Melanoma': 0.0003, 'PNST': 0.9966, 'SCC': 0.0028, 'Trichoblastoma': 0.0003}
Slide Classification:  PNST
